In [ ]:
#Resizing input data
import numpy as np
import itertools
import matplotlib.pyplot as plt

def resize_data(data):
    initial_size_x = data.shape[1]
    initial_size_y = data.shape[2]
    initial_size_z = data.shape[3]

    new_size_x = 16
    new_size_y = 128
    new_size_z = 128
    delta_x = initial_size_x / new_size_x
    delta_y = initial_size_y / new_size_y
    delta_z = initial_size_z / new_size_z

    new_data = np.zeros((data.shape[0], new_size_x, new_size_y, new_size_z))

    for i, x, y, z in itertools.product(range(data.shape[0]),
                                       range(new_size_x),
                                       range(new_size_y),
                                       range(new_size_z)):
        new_data[i][x][y][z] = data[i, int(x * delta_x), int(y * delta_y), int(z * delta_z)]

    return new_data

In [ ]:
#Importing Train Data
trainTargets = pd.read_excel(....)
trainTargets = trainTargets.values.tolist()
trainTargets=np.array(trainTargets)
traindata=np.load('.....npy')

# Resize Train Data
traindata= resize_data(traindata)


In [ ]:
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

data=traindata
labels=trainTargets
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define lists to store the results
val_AUC=[]
val2_loss=[]
val_acc=[]
val_recal=[]
val2_precision=[]

train_AUC=[]
train2_loss=[]
train_acc=[]
train_recal=[]
train2_precision=[]

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)
num_classes = 2

for train_index, val_index in skf.split(data, labels):
    # Split the data into training and validation sets
    x_train, x_val = data[train_index], data[val_index]
    y_train, y_val = labels[train_index], labels[val_index]


    x_train = np.expand_dims(x_train, axis=-1)
    x_val = np.expand_dims(x_val, axis=-1)

    # Calculate weights using sklearn for imbalanced data
    yy = y_train.flatten()

    class_weights = compute_class_weight('balanced', classes=np.unique(yy), y=yy)
    class_weights_dict = dict(enumerate(class_weights))
    print(class_weights_dict)

    y_train2 = to_categorical(y_train, num_classes)
    y_val2 = to_categorical(y_val, num_classes)

    initial_learning_rate = 0.00001
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=10000, decay_rate=0.50, staircase=True
    )


    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[
            tf.keras.metrics.CategoricalAccuracy(),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recal'),
            tf.keras.metrics.AUC(name='auc')])

    # Train the model with early stopping
    batch_size =50
    epochs =20
    print(x_train.shape)
    history = model.fit(x_train, y_train2, batch_size=batch_size, epochs=epochs,
                    validation_data=(x_val, y_val2), class_weight=class_weights_dict,
                    callbacks=[early_stop])

    # Access the training metrics from the history object
    train_acc1 = history.history['categorical_accuracy']
    train_loss1 = history.history['loss']
    train_precision1 = history.history['precision']
    train_recal1 = history.history['recal']
    train_AUC1 = history.history['auc']

    # Access the validation metrics from the history object
    val_acc1 = history.history['val_categorical_accuracy']
    val_loss1 = history.history['val_loss']
    val_precision1 = history.history['val_precision']
    val_recal1 = history.history['val_recal']
    val_AUC1 = history.history['val_auc']

    # Plotting the training and validation accuracy over epochs
    plt.plot(train_acc1, label='Training Accuracy')
    plt.plot(val_acc1, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    # Plotting the training and validation accuracy over epochs
    plt.plot(train_loss1, label='Training Loss')
    plt.plot(val_loss1, label='Validation Loss')
    plt.title('Training and Validation Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    # Plotting AUC
    if 'auc' in history.history:
        plt.plot(history.history['auc'])
        plt.plot(history.history['val_auc'])
        plt.title('Model AUC')
        plt.ylabel('AUC')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()
    else:
        print("AUC not found in history. Make sure 'AUC' is included as a metric in model compilation.")

    # Evaluate the model on the training and validation sets
    a=model.evaluate(x_train, y_train2, verbose=1)
    train_loss, train_accuracy, train_precision, train_recall, train_auc= model.evaluate(x_train, y_train2, verbose=1)
    val_loss, val_accuracy, val_precision, val_recall, val_auc= model.evaluate(x_val, y_val2, verbose=1)
    print(a)

    # Append the results to the lists
    train_acc.append(train_accuracy)
    train_AUC.append(train_auc)
    train2_loss.append(train_loss)
    train_recal.append(train_recall)
    train2_precision.append(train_precision)

    val_acc.append(val_accuracy)
    val_AUC.append(val_auc)
    val2_loss.append(val_loss)
    val_recal.append(val_recall)
    val2_precision.append(val_precision)

loss_max=max(val2_loss)
loss_ave=np.mean(val_loss)
loss_std=np.std(val_loss)

acc_max=max(val_acc)
acc_ave=np.mean(val_acc)
acc_std=np.std(val_acc)

prec_max=max(val2_precision)
prec_ave=np.mean(val2_precision)
prec_std=np.std(val2_precision)

recal_max=max(val_recal)
recal_ave=np.mean(val_recal)
recal_std=np.std(val_recal)

AUC_max=max(val_AUC)
AUC_ave=np.mean(val_AUC)
AUC_std=np.std(val_AUC)


print('val_loss=', loss_ave,"+-", loss_std, "MAX=", loss_max)
print('val_acc=', acc_ave,"+-", acc_std, "MAX=", acc_max)
print('val_AUC=', AUC_ave,"+-", AUC_std, "MAX=", AUC_max)
print('val_precision=', prec_ave,"+-", prec_std, "MAX=", prec_max)
print('val_recal=', recal_ave,"+-", recal_std, "MAX=", recal_max)

train_loss_max=max(train2_loss)
train_loss_ave=np.mean(train2_loss)
train_loss_std=np.std(train2_loss)

train_acc_max=max(train_acc)
train_acc_ave=np.mean(train_acc)
train_acc_std=np.std(train_acc)

train_prec_max=max(train2_precision)
train_prec_ave=np.mean(train2_precision)
train_prec_std=np.std(train2_precision)

train_recal_max=max(train_recal)
train_recal_ave=np.mean(train_recal)
train_recal_std=np.std(train_recal)

train_AUC_max=max(train_AUC)
train_AUC_ave=np.mean(train_AUC)
train_AUC_std=np.std(train_AUC)


print('train_loss=', train_loss_ave,"+-", train_loss_std, "MAX=", train_loss_max)
print('train_acc=', train_acc_ave,"+-", train_acc_std, "MAX=", train_acc_max)
print('train_AUC=', train_AUC_ave,"+-", train_AUC_std, "MAX=", train_AUC_max)
print('train_precision=', train_prec_ave,"+-", train_prec_std, "MAX=", train_prec_max)
print('train_recal=', train_recal_ave,"+-", train_recal_std, "MAX=", train_recal_max)


# Calculate the average training and validation accuracies across folds
avg_train_acc = sum(train_acc) / len(train_acc)
print("Average training accuracy: ", avg_train_acc)

################################
c=(time.time() - start_time)/60
print('training time (min)=', c)


In [ ]:
!pip install pyyaml h5py
# save model
model.save('......h5')


In [ ]:
### Importing Test Data ####

testTargets = pd.read_excel(....)
testTargets = testTargets.values.tolist()
testTargets=np.array(testTargets)
testdata=np.load(r'.....npy')
# Resize test data
testdata= resize_data(testdata)


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import brier_score_loss, roc_auc_score
from scipy.stats import norm

# Prepare test data
x_test = testdata  # Assuming testdata is in shape (num_samples, 256, 256, 15)
y_test = testTargets  # Assuming testTargets is in shape (num_samples,)
x_test = np.expand_dims(x_test, axis=-1)  # Add channel dimension to match model input shape (num_samples, 256, 256, 15, 1)
y_test2 = to_categorical(y_test, num_classes=2)  # Convert y_test to one-hot encoding for model evaluation

# Evaluate model u
TEST = model.evaluate(x_test, y_test2, batch_size=5, verbose=1)
test_loss, test_accuracy, test_precision, test_recall, test_auc = TEST


# Predict probabilities for calculating additional metrics
y_pred_probs = model.predict(x_test)
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Flatten y_test only if it's 2D
# y_test = np.argmax(y_test2, axis=1)

# Calculate the Brier Score
brsc = brier_score_loss(y_test, y_pred_probs[:, 1])
###########################
from sklearn.metrics import confusion_matrix

# Predict labels and probabilities
y_pred_labels = np.argmax(y_pred_probs, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
tn, fp, fn, tp = cm.ravel()  # Flatten confusion matrix

# Calculate Precision, Recall, F1 Score, and Specificity
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall) if (test_precision + test_recall) > 0 else 0

# Function to calculate 95% confidence intervals
def confidence_interval(metric, n, z=1.96):
    return z * np.sqrt((metric * (1 - metric)) / n)

# Calculate confidence intervals
acc_ci = confidence_interval(test_accuracy, len(y_test))
pre_ci = confidence_interval(test_precision, len(y_test))
rec_ci = confidence_interval(test_recall, len(y_test))
auc_ci = confidence_interval(test_auc, len(y_test))
f1_ci = confidence_interval(f1, len(y_test))
spec_ci = confidence_interval(specificity, len(y_test))

# Print results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ± {acc_ci:.4f}")
print(f"Test Precision: {test_precision:.4f} ± {pre_ci:.4f}")
print(f"Test Recall: {test_recall:.4f} ± {rec_ci:.4f}")
print(f"Test F1 Score: {f1:.4f} ± {f1_ci:.4f}")
print(f"Test Specificity: {specificity:.4f} ± {spec_ci:.4f}")
print(f"Test AUC: {test_auc:.4f} ± {auc_ci:.4f}")
print(f"Brier Score: {brsc:.4f}")
